In [1]:
import sys, os, glob, warnings, logging
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sw_plotting import change_bar_width
from sw_utilities import tukeyTest

# logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

# ignore warnings
warnings.filterwarnings('ignore')

# plotting configuration
font = {'family' : 'Arial',
        'size'   : 7}
matplotlib.rc('font', **font)
plt.rcParams['svg.fonttype'] = 'none'

# Make a folder if it is not already there to store exported figures
!mkdir ../jupyter_figures


mkdir: ../jupyter_figures: File exists


In [2]:
def get_linescan_data(folder, sgRNA, time_d, n_edges=5):
    '''
    Read in linescan data and annotate with meta data
    
    Inputs:
    -------
        - folder: the folder that contains the linescan output csv file
        - sgRNA: which sgRNA was used for the treatment of this group
        - time_d: how many days after culture the samples were collected
        - n_edges: how many edges were quantified per sample
        
    Outputs:
        - df: pandas data frame with linescan data and annotation data
        
    '''
    
    linescan_files = glob.glob(os.path.join(folder, '*.txt'))

    if len(linescan_files) > 1:
        print('There are more than 1 output file in', folder)
        print('The newest one will be used.')
        linescan_files = sorted(linescan_files,
                                key = os.path.getmtime)
        
    df = pd.read_csv(linescan_files[-1])
    
    # annotate data
    assert len(df) % 5 == 0
    n_organoids = int(len(df) / 5)
    
    sample_ids = []
    organoid_ids = []
    for i in range(n_organoids):
        for j in range(n_edges):
            sample_ids.append(sgRNA + '_' + str(i+1))
            organoid_ids.append( str(i+1) )
    
    df['sample_id'] = sample_ids
    df['organoid_id'] = organoid_ids
    df['sgRNA'] = [sgRNA]*len(df)
    df['time_d'] = time_d
    df['groups'] = [sgRNA + '_' + str(time_d)]*len(df)
    
    return df

In [3]:
input_folder = '../data/220606-a3int-IF'
folders = glob.glob(os.path.join(input_folder, '*line-scan-output' + os.path.sep))

In [4]:
dfs = []
for folder in folders:
    if folder.endswith(os.path.sep):
        folder = folder[:-1]
    sgRNA = os.path.basename(folder).split('-')[1]
    time_d = 4
    dfs.append(get_linescan_data(folder, sgRNA, time_d, n_edges=5))

In [5]:
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [6]:
df.tail()

,file_name,edge_category,id,a3int_AUC,a3int_peak_height,NLSmSL_cell_1,NLSmSL_cell_2,z_position,sample_id,organoid_id,sgRNA,time_d,groups
80,4,High_High,1,1614.4828,434.1611,100.2405,141.9268,1,319_4,4,319,4,319_4
81,4,High_High,2,-183.6052,168.7956,161.0382,352.6743,1,319_4,4,319,4,319_4
82,4,High_High,3,843.5868,295.6452,227.1358,260.8445,1,319_4,4,319,4,319_4
83,4,High_High,4,1728.4787,235.3561,426.1702,309.2403,1,319_4,4,319,4,319_4
84,4,High_High,5,4775.3020,761.2314,256.4160,226.0904,1,319_4,4,319,4,319_4


In [7]:
# # normalize to the average of control
# ctrl_average = df[df.sgRNA=='188'].a3int_peak_height.mean()
# df['a3int_normed'] = df['a3int_peak_height'] / ctrl_average

In [9]:
df_grouped = df.groupby('sample_id').describe()['a3int_peak_height'].reset_index()
df_grouped['sgRNA'] = [i[:-2] for i in df_grouped.sample_id]

In [10]:
ctrl_average = df_grouped[df.sgRNA=='188']['mean'].mean()
df_grouped['a3int_normed'] = df_grouped['mean'] / ctrl_average 

In [12]:
# normalize to the average of control
# ctrl_average = df[df.sgRNA=='188'].a6int_peak_height.mean()
df['a3int_normed'] = df['a3int_peak_height'] / ctrl_average

In [14]:
# Swarm plot of 4 categories

outputPrefix = '220606-IF-a3-integrin-sg-Itga9-d4-grouped-bar-swarm'
outputFigPath = "../jupyter_figures/" + outputPrefix + ".svg"

fig_width = 1.2
fig_height = 1.8

fig = plt.figure(figsize=(fig_width,fig_height), dpi=300)
# ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax = fig.add_axes([0.35, 0.4, 0.6, 0.55])

ax = sns.swarmplot(x='sgRNA', y='a3int_normed', data=df,
                   hue='organoid_id',
#                    palette={'1': 'red', '2': 'green', '3': 'blue', '4': yellow', '5': 'purple'},
                   # color="blue",
                   alpha=0.5,
                   size=1.0 )

ax = sns.barplot(x='sgRNA', y='a3int_normed', data=df_grouped,
                 color=".7", alpha=1.0,
                 errwidth=.7, errcolor="k", capsize=.2, ci='sd')

# plt.ylim(0, 1.7)
# plt.ylim(0, 2.7)
# plt.yticks([0, 3, 6])

plt.xlabel(None)
# plt.ylabel(r"$\beta$1-integrin intensity")
plt.ylabel('ITGA3 intensity')

# make the bar width narrower
change_bar_width(ax, .5)

# rotate x tick labels
# x_labels = ax.get_xticklabels()
x_labels = ['sg-Control', 'sg1-Itga9', 'sg2-Itga9', 'sg3-Itga9']
ax.set_xticklabels(labels=x_labels, rotation=45, ha="right")

ax.get_legend().remove()

for o in fig.findobj():
    o.set_clip_on(False)
for o in ax.findobj():
    o.set_clip_on(False)

if outputFigPath is not None:
    plt.savefig(outputFigPath)


In [16]:
tukeyTest(df.a3int_normed, df.sgRNA)

Summary of test:
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   188    317  -0.4081 0.0064 -0.7268 -0.0894   True
   188    318  -0.1418 0.6321 -0.4605  0.1769  False
   188    319  -0.4331 0.0034 -0.7518 -0.1144   True
   317    318   0.2663 0.1687 -0.0696  0.6023  False
   317    319   -0.025    0.9 -0.3609   0.311  False
   318    319  -0.2913 0.1127 -0.6272  0.0446  False
----------------------------------------------------
p values of all pair-wise tests:
 [0.00643324 0.63213169 0.00337872 0.16866405 0.9        0.11271531]


In [17]:
tukeyTest(df_grouped.a3int_normed, df_grouped.sgRNA)

Summary of test:
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   188    317  -0.4081 0.0512 -0.8181  0.0019  False
   188    318  -0.1418 0.7259 -0.5518  0.2682  False
   188    319  -0.4331 0.0372  -0.843 -0.0231   True
   317    318   0.2663 0.3129 -0.1658  0.6985  False
   317    319   -0.025    0.9 -0.4571  0.4072  False
   318    319  -0.2913 0.2452 -0.7234  0.1409  False
----------------------------------------------------
p values of all pair-wise tests:
 [0.05121097 0.72586557 0.03716786 0.31286335 0.9        0.24518577]


In [16]:
for i in df.sgRNA.unique():
    print(i, 'n =', len(df[df.sgRNA==i]))

188 n = 25
317 n = 20
318 n = 20
319 n = 20
